# How to use pretrained finetuned model

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece #some pre-trained models require this tokenized

     |████████████████████████████████| 3.4 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 60.6 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 1.2 MB 8.3 MB/s 


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import os
import torch

In [ ]:
!gdown --id '1-4yzXqT8mPwF6jdw-H-a_S1RL61_v6o5'

In [ ]:
qas_t1 = pd.read_json('train.json')
qas_t1.head()

,context,qas
0,โอตสึ โอตสึ () เป็นเมืองหลักของจังหวัดชิงะ ประ...,"[{'id': 1, 'question': 'เมืองโอตสึเคยเป็นเมือง..."
1,โอเปก องค์การกลุ่มประเทศผู้ส่งน้ำมันออก () หรื...,"[{'id': 4, 'question': 'องค์การกลุ่มประเทศผู้ส..."
2,โอเปก องค์การกลุ่มประเทศผู้ส่งน้ำมันออก () หรื...,"[{'id': 4, 'question': 'องค์การกลุ่มประเทศผู้ส..."
3,ทะเลคารา ทะเลคารา (; ) เป็นส่วนหนึ่งของมหาสมุท...,"[{'id': 18, 'question': 'ทะเลคาราซึ่งอยู่ทางตอ..."
4,ทะเลคารา ทะเลคารา (; ) เป็นส่วนหนึ่งของมหาสมุท...,"[{'id': 18, 'question': 'ทะเลคาราซึ่งอยู่ทางตอ..."


In [ ]:
qas_t1.iloc[0].qas

[{'answers': [{'answer_start': 183, 'text': 'สมเด็จพระจักรพรรดิเท็นจิ'}],
  'id': 1,
  'is_impossible': False,
  'question': 'เมืองโอตสึเคยเป็นเมืองหลวงของญี่ปุ่นในช่วง พ.ศ. 1210–1215 ในรัชสมัยของพระจักรพรรดิองค์ใด'}]

In [ ]:
# Load pretrained from Hugging Face
pretrained = "deepset/xlm-roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(pretrained)

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

# Version 1: Manual

In [ ]:
def answer_question(question, context, tokenizer=tokenizer, model=model):
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    try:
        answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer_ids = input_ids[answer_start:answer_end]
        answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
        answer = tokenizer.convert_tokens_to_string(answer_tokens)

        answer_score = torch.max(answer_start_scores)

        if answer.strip(): # If it can be stripped -> its type: str
            return answer,  answer_score
        else:
            return "<UNK/>", -1
    except Exception as e:
        print(e)
        return "<UNK/>", -1

In [ ]:
question = qas_t1.loc[1, 'qas'][0]['question']
correct_answer = qas_t1.loc[1, 'qas'][0]['answers'][0]['text']
context = qas_t1.loc[1, 'context']

context, question, correct_answer

('โอเปก องค์การกลุ่มประเทศผู้ส่งน้ำมันออก () หรือ โอเปก () กำเนิดใน พ.ศ. 2503 มีสมาชิกก่อตั้ง 5 ประเทศ ประกอบด้วยประเทศซาอุดีอาระเบีย อิรัก อิหร่าน คูเวต และเวเนซุเอลา เดิมมีสำนักงานใหญ่ที่ประเทศสวิตเซอร์แลนด์ ปัจจุบัน โอเปกมีสมาชิกเพิ่มอีก 9 ประเทศ คือ สหรัฐอาหรับเอมิเรตส์ กาตาร์ ไนจีเรีย แอลจีเรีย ลิเบีย กาบอง (ต่อมาถอนตัวในปี ค.ศ. 2008 แล้วกลับมาในปี ค.ศ. 2016) เอกวาดอร์ (เคยเป็นสมาชิกในปี ค.ศ. 1973 แล้วถอนตัวไป แล้วกลับมาในปี ค.ศ. 2007) แองโกลา และล่าสุดอิเควทอเรียลกินี รวมเป็น 14 ประเทศ โดยได้ย้ายสำนักงานใหญ่ไปที่กรุงเวียนนา ประเทศออสเตรีย ตั้งแต่ ค.ศ. 1965\n',
 'องค์การกลุ่มประเทศผู้ส่งน้ำมันออกหรือโอเปกซึ่งถือกำเนิดในปี พ.ศ. 2503 มีสมาชิกร่วมก่อตั้งกี่ประเทศ',
 '5 ประเทศ')

In [ ]:
predict, score = answer_question(question, context)

In [ ]:
predict, score

('5 ประเทศ', tensor(4.2495, grad_fn=<MaxBackward1>))

# Version 2: Pipeline

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
print(pipe(question=question, context=context))

{'score': 0.5123615860939026, 'start': 91, 'end': 100, 'answer': ' 5 ประเทศ'}
